<a href="https://colab.research.google.com/github/FeDextroyer/MQTT/blob/master/Sample%20Extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install paho-mqtt ipywidgets

import paho.mqtt.client as paho
import os
import json
import time
from datetime import datetime
import ipywidgets as widgets

ACCESS_TOKEN='olamxoZMg9cr3IPirFYH'
broker="demo.thingsboard.io"
port=1883
sensor_data = {'temperature': 0, 'humidity': 0, 'pressure': 0, 'luminosity': 0}

def on_publish(client,userdata,result):
    print("data published to thingsboard \n")
    pass

def update_sensor_data(change):
    sensor_data[change['owner'].description] = change['new']
    print(sensor_data)

    payload = json.dumps(sensor_data)
    ret = client1.publish("v1/devices/me/telemetry", payload)
    print("Please check LATEST TELEMETRY field of your device")
    print(payload)

temperature_slider = widgets.FloatSlider(min=0, max=100, description='temperature', layout=widgets.Layout(width='50%'))
humidity_slider = widgets.FloatSlider(min=0, max=100, description='humidity', layout=widgets.Layout(width='50%'))
pressure_slider = widgets.FloatSlider(min=0, max=100, description='pressure', layout=widgets.Layout(width='50%'))
luminosity_slider = widgets.FloatSlider(min=0, max=100, description='luminosity', layout=widgets.Layout(width='50%'))

temperature_slider.observe(update_sensor_data, names='value')
humidity_slider.observe(update_sensor_data, names='value')
pressure_slider.observe(update_sensor_data, names='value')
luminosity_slider.observe(update_sensor_data, names='value')

display(temperature_slider)
display(humidity_slider)
display(pressure_slider)
display(luminosity_slider)

client1= paho.Client("control1")
client1.on_publish = on_publish
client1.username_pw_set(ACCESS_TOKEN)
client1.connect(broker,port,keepalive=60)

while True:
    client1.loop(0.1)
    time.sleep(3)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


FloatSlider(value=0.0, description='temperature', layout=Layout(width='50%'))

FloatSlider(value=0.0, description='humidity', layout=Layout(width='50%'))

FloatSlider(value=0.0, description='pressure', layout=Layout(width='50%'))

FloatSlider(value=0.0, description='luminosity', layout=Layout(width='50%'))

KeyboardInterrupt: ignored

{'temperature': 42.0, 'humidity': 0, 'pressure': 0, 'luminosity': 0}
Please check LATEST TELEMETRY field of your device
{"temperature": 42.0, "humidity": 0, "pressure": 0, "luminosity": 0}
{'temperature': 42.0, 'humidity': 6.2, 'pressure': 0, 'luminosity': 0}
Please check LATEST TELEMETRY field of your device
{"temperature": 42.0, "humidity": 6.2, "pressure": 0, "luminosity": 0}
{'temperature': 42.0, 'humidity': 6.2, 'pressure': 24.9, 'luminosity': 0}
Please check LATEST TELEMETRY field of your device
{"temperature": 42.0, "humidity": 6.2, "pressure": 24.9, "luminosity": 0}
{'temperature': 42.0, 'humidity': 6.2, 'pressure': 24.9, 'luminosity': 69.1}
Please check LATEST TELEMETRY field of your device
{"temperature": 42.0, "humidity": 6.2, "pressure": 24.9, "luminosity": 69.1}
